<br/>
<span style="font-size: 4em;">MSP Population Synthesis Pipeline</span>
<br/><br/>
<span style="font-size: 4em;">for MSIP 2020</span>

Don't actually run this notebook. This is just if you care to see exactly what I did.

In [23]:
import cPickle
import os
import tarfile
import pandas as pd
from glob import glob
from copy import deepcopy
from itertools import product
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from PTAOptimizer import timing_precision
from PTAOptimizer import mkasc
from psrpoppy.populate import rednoise_cs10

# Add Red Noise into Model
Compute $A_\mathrm{red}$ and $\gamma_\mathrm{red}$ for each pulsar in the population.

In [29]:
# read in psrpoppy population model
# This is old model based only on Parkes MB
with open('../models/MSPpop.model', 'rb') as popfile:
    pop = cPickle.load(popfile)

In [30]:
len(pop.population)

38475

In [4]:
pop.rnDistPars = [1.6, -1.4, 1.1, 2.0, 1.6]
for p in pop.population:
    p.rn_amp, p.rn_index = rednoise_cs10(p, lnC2=1.6,
                                         alpha=-1.4,
                                         beta=1.1,
                                         gamma=2.0,
                                         delta=1.6)

# Determine Pulsars to Exclude
Remove pulsars whose RMS white noise $\left(\sqrt{\sigma_\mathrm{RN}^2 + \sigma_\mathrm{j}^2}\right)$ resemble pulsars in the real NANOGrav dataset.

In [7]:
# AO L-S frequency range
Lband_nus = np.linspace(1.44 - .618 / 2, 1.44 + .618 / 2, 55 + 1)[:-1]
# S-high
Shi_nus = np.linspace(2.227 - .354 / 2, 2.227 + .354 / 2, 30 + 1)[:-1]
# S-low
Slo_nus = np.linspace(1.79 - .180 / 2, 1.79 + .180 / 2, 15 + 1)[:-1]
aoLS_nus = np.sort(np.concatenate([Lband_nus, Slo_nus, Shi_nus]))

# AO 430-L frequency range
nus_ao430 = np.linspace(.432 - .02 / 2, .432 + .02 / 2, 10 + 1)[:-1]
nus_aoL = np.linspace(1.44 - .58 / 2, 1.44 + .58 / 2, 90 + 1)[:-1]
ao430L_nus = np.concatenate([nus_ao430, nus_aoL])

# GB 800-1200 frequency range
nus_gb800 = np.linspace(.820 - .200 / 2, .820 + .200 / 2, 20 + 1)[:-1]
nus_gb1_2 = np.linspace(1.510 - .800 / 2, 1.510 + .800 / 2, 80 + 1)[:-1]
gbt80012_nus = np.concatenate([nus_gb800, nus_gb1_2])

In [8]:
# Time population with each configuration of AO and GBT frequency
# It's 30 min/frequency right?
gbt80012_pta = timing_precision.run_timing(pop, gbt80012_nus, 
                                             rxspecfile='GBT_Rcvr_800-Rcvr_1_2_test.txt',
                                             t_int=3600., dec_lim=(90.0, -46.0)) 
ao430L_pta = timing_precision.run_timing(pop, ao430L_nus, 
                                            rxspecfile='AO_430_Lwide_test.txt',
                                            t_int=3600., dec_lim=(39.0, 0.0))
aoLS_pta = timing_precision.run_timing(pop, aoLS_nus, 
                                        rxspecfile='AO_Lwide_Swide.txt',
                                        t_int=3600., dec_lim=(39.0, 0.0))

In [9]:
# Pick best frequency range for each MSP at Arecibo
aobest_list = timing_precision.best_timed([ao430L_pta, aoLS_pta])
print('{} MSPs in sim AO list'.format(len(aobest_list)))
# Exclude AO sky from GBT
gbt_list = [p for p in gbt80012_pta.population if (p.dec >= 39. or p.dec <= 0.) and p.sigma_toa > 0.]
print('{} MSPs in sim GBT list'.format(len(gbt_list)))

1812 MSPs in sim AO list
9578 MSPs in sim GBT list


In [10]:
# Read in NG RMSs and separate by telescope
df = pd.read_csv('NG_RMS.csv')
ngrms_gbt = list(np.array(df.RMS)[np.where(df.Observatory == 'GBT')])
ngrms_ao = list(np.array(df.RMS)[np.where(df.Observatory == 'AO')])

In [11]:
# Remove simulated MSPs similar to NG MSPs from GBT PTA
gbt_excl = []
for i in range(len(ngrms_gbt)):
    closest = sorted(product(gbt_list, ngrms_gbt),
                       key=lambda t: abs( np.sqrt(t[0].sigma_rn ** 2 + t[0].sigmaj(3600.)**2) - t[1]))[0]
    gbt_list.remove(closest[0])
    gbt_excl.append(closest[0])
    ngrms_gbt.remove(closest[1])
print('{} MSPs in sim GBT list, {} MSPs in NG RMS list'.format(len(gbt_list), len(ngrms_gbt)))

9544 MSPs in sim GBT list, 0 MSPs in NG RMS list


In [12]:
# get location of excluded pulsars in GBT pta
gbt_exclidx = [gbt80012_pta.population.index(p) for p in gbt_excl]

In [13]:
# Remove simulated MSPs similar to NG MSPs from AO PTA
ao_excl = []
for i in range(len(ngrms_ao)):
    closest = sorted(product(aobest_list, ngrms_ao),
                       key=lambda t: abs( np.sqrt(t[0].sigma_rn ** 2 + t[0].sigmaj(3600.)**2) - t[1]))[0]
    aobest_list.remove(closest[0])
    ao_excl.append(closest[0])
    ngrms_ao.remove(closest[1])
print('{} MSPs in sim AO list, {} MSPs in NG RMS list'.format(len(aobest_list), len(ngrms_ao)))

1777 MSPs in sim AO list, 0 MSPs in NG RMS list


In [14]:
# get location of excluded pulsars in AO pta
ao_exclidx = []
for p in ao_excl:
    try:
        ao_exclidx.append(ao430L_pta.population.index(p))
    except ValueError:
        ao_exclidx.append(aoLS_pta.population.index(p))

In [15]:
# Remove excluded pulsars from population and save model to file
all_exclidx = set(ao_exclidx + gbt_exclidx)
incl_pop = deepcopy(pop)
incl_pop.population = [p for i, p in enumerate(pop.population) if i not in all_exclidx]
with open('models/incl_MSPpop.model', 'wb') as inclf:
    cPickle.dump(incl_pop, inclf)

In [16]:
# Save excluded pulsars (FOpPulsar's) to a separate model file
excl_pop = deepcopy(pop)
excl_pop.population = ao_excl + gbt_excl
with open('models/excl_MSPpop.model', 'wb') as exclf:
    cPickle.dump(excl_pop, exclf)

# Time Populations with Respective Instruments
Run included and excluded pulsars through various telescope configurations and compute $\sigma$s.

In [17]:
# set up frequency ranges

# AO L-S frequency range
Lband_nus = np.linspace(1.44 - .618 / 2, 1.44 + .618 / 2, 55 + 1)[:-1]
# S-high
Shi_nus = np.linspace(2.227 - .354 / 2, 2.227 + .354 / 2, 30 + 1)[:-1]
# S-low
Slo_nus = np.linspace(1.79 - .180 / 2, 1.79 + .180 / 2, 15 + 1)[:-1]
aoLS_nus = np.sort(np.concatenate([Lband_nus, Slo_nus, Shi_nus]))

# AO 430-L frequency range
nus_ao430 = np.linspace(.432 - .02 / 2, .432 + .02 / 2, 10 + 1)[:-1]
nus_aoL = np.linspace(1.44 - .58 / 2, 1.44 + .58 / 2, 90 + 1)[:-1]
ao430L_nus = np.concatenate([nus_ao430, nus_aoL])

# GB 800-1200 frequency range
nus_gb800 = np.linspace(.820 - .200 / 2, .820 + .200 / 2, 20 + 1)[:-1]
nus_gb1_2 = np.linspace(1.510 - .800 / 2, 1.510 + .800 / 2, 80 + 1)[:-1]
gbt80012_nus = np.concatenate([nus_gb800, nus_gb1_2])

# GBT UWB 
gbuwb_ctrfreq = 2.35 #GHz
gbuwb_bw = 3.3 # GHz
gbuwb_nus = np.linspace(gbuwb_ctrfreq - gbuwb_bw / 2,
                        gbuwb_ctrfreq + gbuwb_bw / 2,
                        100 + 1)[:-1]

# AO UWB
aouwb_ctrf = 2.368
aouwb_bw = 3.328
aouwb_nus = np.linspace(aouwb_ctrf - aouwb_bw / 2, aouwb_ctrf + aouwb_bw / 2, 100 + 1)[:-1]

# DSA 0.7-2 GHz
dsa072_nus = np.linspace(1.35 - 1.3 / 2, 1.35 + 1.3 / 2, 100 + 1)[:-1]

# VLA L & S 
vlaL_nus = np.linspace(1.5 - 1. / 2., 1.5 + 1. / 2., 50 + 1)[:-1]
vlaS_nus = np.linspace(3. - 2. / 2., 3. + 2. / 2., 50 + 1)[:-1]
vlaLS_nus = np.concatenate([vlaL_nus, vlaS_nus])

# ngVLA Band 1 and 2
ngvB1_nus = np.linspace(2.35 - 2.3 / 2, 2.35 + 2.3 / 2, 50 + 1)[:-1]
ngvB2_nus = np.linspace(7.9 - 8.8 / 2, 7.9 + 8.8 / 2, 50 + 1)[:-1]
ngvlaB12_nus = np.concatenate([ngvB1_nus, ngvB2_nus])

In [ ]:
# Run included and excluded ~real MSPs through timing campaigns
for cl in ['excl', 'incl']:
    with open('models/{}_MSPpop.model'.format(cl), 'rb') as popfile:                     
        clpop = cPickle.load(popfile)  

    # Arecibo 430 & Lwide PTA
    ao430L_pta = timing_precision.run_timing(clpop, ao430L_nus,  
                                             rxspecfile='AO_430_Lwide_test.txt',
                                             t_int=3600.,
                                             dec_lim=(39.0, 0.0))
    with open('models/{}_AO_430_Lwide_V3.pta.model'.format(cl), 'wb') as aooutf:
        cPickle.dump(ao430L_pta, aooutf)
        
    # Arecibo L & S-band
    aoLS_pta = timing_precision.run_timing(clpop, aoLS_nus,  
                                             rxspecfile='AO_Lwide_Swide.txt',
                                             t_int=3600.,
                                             dec_lim=(39., 0.))
    with open('models/{}_AO_Lwide_Swide_V3.pta.model'.format(cl), 'wb') as aooutf:
        cPickle.dump(aoLS_pta, aooutf)

    # GBT PTA
    gbt_pta = timing_precision.run_timing(clpop, gbt80012_nus, 
                                          rxspecfile='GBT_Rcvr_800-Rcvr_1_2_test.txt',
                                          t_int=3600.,
                                          dec_lim=(90.0, -46.0))
    with open('models/{}_GBT_Rcvr800_Rcvr12_V2.pta.model'.format(cl), 'wb') as gboutf:        
        cPickle.dump(gbt_pta, gboutf)

    # GBT UWB (double integration time for wide-band)
    gbuwbspecfile = 'Corr_Efficiency_Dielectric_Loaded_Dec6_lowfmod.txt'
    gbtuwb_pta = timing_precision.run_timing(clpop, gbuwb_nus, 
                                             rxspecfile=gbuwbspecfile,
                                             t_int=3600., dec_lim=(90, -46))
    with open('models/{}_GBT_UWB_V3.pta.model'.format(cl), 'wb') as guwboutf:   
         cPickle.dump(gbtuwb_pta, guwboutf)

    # AO UWB
    aouwb_pta = timing_precision.run_timing(clpop, aouwb_nus, 
                                            rxspecfile='AO_UWB.txt',
                                            t_int=3600., dec_lim=(39., 0.))
    with open('models/{}_AO_UWB_V1.pta.model'.format(cl), 'wb') as auwboutf:   
         cPickle.dump(aouwb_pta, auwboutf)

    # DSA 0.7-2 GHz full array
    dsa072_pta = timing_precision.run_timing(clpop, dsa072_nus,
                                             Trx=20.,
                                             Gain=10.,
                                             eps=0.01,
                                             t_int=3600.,
                                             dec_lim=(90.0, -44.0))
    with open('models/{}_dsa2k_07-2_V3.pta.model'.format(cl), 'wb') as dsa072outf:
        cPickle.dump(dsa072_pta, dsa072outf)

    # DSA 0.7-2, 1/2 Aeff
    dsa072half_pta = timing_precision.run_timing(clpop, dsa072_nus,
                                                 Trx=20.,
                                                 Gain=5.,
                                                 eps=0.01, t_int=3600.,
                                                 dec_lim=(90.0, -44.0))
    with open('models/{}_dsa2k_07-2_halfAeff_V3.pta.model'.format(cl), 'wb') as dsahalfoutf:
        cPickle.dump(dsa072half_pta, dsahalfoutf)

    # DSA 0.7-2, 1/4 Aeff
    dsa072quart_pta = timing_precision.run_timing(clpop, dsa072_nus,
                                                 Trx=20.,
                                                 Gain=2.5,
                                                 eps=0.01, t_int=3600.,
                                                 dec_lim=(90.0, -44.0))
    with open('models/{}_dsa2k_07-2_quartAeff_V3.pta.model'.format(cl), 'wb') as dsaqtoutf:
        cPickle.dump(dsa072quart_pta, dsaqtoutf)

    # VLA L-S band
    vlaLS_pta = timing_precision.run_timing(clpop, vlaLS_nus,
                                            rxspecfile='VLA_L-SBand.txt',
                                            t_int=3600.,
                                            dec_lim=(90.0, -44.0))
    with open('models/{}_VLA_LBand-SBand_V2.pta.model'.format(cl), 'wb') as vlaLSf:
        cPickle.dump(vlaLS_pta, vlaLSf)


    # ngVLA Band1-Band2 20% area
    ngvla_pta = timing_precision.run_timing(clpop, ngvlaB12_nus,
                                            rxspecfile='ngVLA_20pct.txt',
                                            t_int=3600.,
                                            dec_lim=(90.0, -44.0))
    with open('models/{}_ngVLA_Band1-2_V3.pta.model'.format(cl), 'wb') as ngvlaf:
        cPickle.dump(ngvla_pta, ngvlaf)

# Write out .par and ASCII files
Write out a .par file for each pulsar and an ASCII file for each pta.model file

In [28]:
from glob import glob

for f in glob('models/*.pta.model'):                                                                                  
    print(f)
    try:
        mkasc.mkasc(f, sortsigma=True)
    except AttributeError:
        print(f + ' didnt have the thing')
        raise AttributeError

models/excl_AO_430_Lwide_V3.pta.model
models/excl_AO_Lwide_Swide_V3.pta.model
models/excl_GBT_Rcvr800_Rcvr12_V2.pta.model
models/excl_GBT_UWB_V3.pta.model
models/excl_AO_UWB_V1.pta.model
models/excl_dsa2k_07-2_V3.pta.model
models/excl_dsa2k_07-2_halfAeff_V3.pta.model
models/excl_dsa2k_07-2_quartAeff_V3.pta.model
models/excl_VLA_LBand-SBand_V2.pta.model
models/excl_ngVLA_Band1-2_V3.pta.model
models/incl_AO_430_Lwide_V3.pta.model
models/incl_AO_Lwide_Swide_V3.pta.model
models/incl_GBT_Rcvr800_Rcvr12_V2.pta.model
models/incl_GBT_UWB_V3.pta.model
models/incl_AO_UWB_V1.pta.model
models/incl_dsa2k_07-2_V3.pta.model
models/incl_dsa2k_07-2_halfAeff_V3.pta.model
models/incl_dsa2k_07-2_quartAeff_V3.pta.model
models/incl_VLA_LBand-SBand_V2.pta.model
models/incl_ngVLA_Band1-2_V3.pta.model


In [ ]:
# write out exclude par files
with open('models/excl_MSPpop.model', 'rb') as exclf:
    exclmod = cPickle.load(exclf)
for p in exclmod.population:
    with open('data/exclude/PAR/{}.par'.format(p.jname()), 'wb') as parf:
        parf.write(p.parfile())

In [ ]:
# write out included par files
with open('models/incl_MSPpop.model', 'rb') as inclf:
    inclmod = cPickle.load(inclf)
for p in inclmod.population:
    with open('data/include/PAR/{}.par'.format(p.jname()), 'wb') as parf:
        parf.write(p.parfile())